In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.autograd import Variable
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from tqdm import trange
from PIL import Image
import argparse
import easydict

%matplotlib inline
custom_style = {'axes.labelcolor': 'white',
                'xtick.color': 'white',
                'ytick.color': 'white'}
sns.set_style("darkgrid", rc=custom_style)
sns.set_context("notebook")
plt.style.use('dark_background')
plt.rcParams["font.size"] = 18

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import MNIST
from torch.utils.data import Dataset
from torch.utils.data import sampler
import contextlib

# PytorchでMNISTを学習

### 参考サイト
https://github.com/pytorch/examples/blob/master/mnist/main.py

In [3]:
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--label_batch_size', type=int, default=32, metavar='N',
#                         help='input batch size for training (default: 32)')
#     parser.add_argument('--unlabel_batch_size', type=int, default=128, metavar='N',
#                         help='input batch size for training (default: 32)')
#     parser.add_argument('--test_batch_size', type=int, default=1000, metavar='N',
#                         help='input batch size for testing (default: 1000)')
#     parser.add_argument('--epoch', type=int, default=10, metavar='N',
#                         help='number of iterations to train (default: 10)')
#     parser.add_argument('--iters', type=int, default=400, metavar='N',
#                         help='number of iterations to train (default: 10000)')
#     parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
#                         help='learning rate (default: 0.01)')
#     parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
#                         help='SGD momentum (default: 0.9)')
#     parser.add_argument('--alpha', type=float, default=1.0, metavar='ALPHA',
#                         help='regularization coefficient (default: 0.01)')
#     parser.add_argument('--xi', type=float, default=10.0, metavar='XI',
#                         help='hyperparameter of VAT (default: 0.1)')
#     parser.add_argument('--eps', type=float, default=1.0, metavar='EPS',
#                         help='hyperparameter of VAT (default: 1.0)')
#     parser.add_argument('--ip', type=int, default=1, metavar='IP',
#                         help='hyperparameter of VAT (default: 1)')
#     parser.add_argument('--workers', type=int, default=8, metavar='W',
#                         help='number of CPU')
#     parser.add_argument('--seed', type=int, default=123, metavar='S',
#                         help='random seed (default: 1)')
#     parser.add_argument('--log-interval', type=int, default=100, metavar='N',
#                         help='how many batches to wait before logging training status')
#     parser.add_argument('--gpu', type=int, default=1, metavar='W',
#                         help='number of CPU')
#     args = parser.parse_args()

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3)
        self.fc1 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(-1, 128)
        x = self.fc1(x)
        return x

        return x

In [5]:
28*28

784

In [6]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()

        self.fc1 = nn.Linear(784, 1200)
#         self.fc2_1 = nn.Linear(1200, 1200)
        self.fc2 = nn.Linear(1200, 600)
#         self.fc2_1 = nn.Linear(600, 300)
#         self.fc2_2 = nn.Linear(300, 150)
        self.fc3 = nn.Linear(600, 10)
#         self.fc3 = nn.Linear(600, 10)
        
        self.bn1 = nn.BatchNorm1d(1200)
        self.bn2 = nn.BatchNorm1d(600)
        self.bn2_1 = nn.BatchNorm1d(300)
        self.bn2_2 = nn.BatchNorm1d(150)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.bn2(x)
#         x = self.fc2_1(x)
#         x = F.relu(x)
#         x = self.bn2_1(x)
#         x = self.fc2_2(x)
#         x = F.relu(x)
#         x = self.bn2_2(x)
        
        x = self.fc3(x)
#         x = F.log_softmax(x, dim=1)
        
        return x

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #畳み込み層を定義する
        #引数は順番に、サンプル数、チャネル数、フィルタのサイズ
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3)
        #フィルタのサイズは正方形であればタプルではなく整数でも可（8行目と10行目は同じ意味）
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        #全結合層を定義する
        #fc1の第一引数は、チャネル数*最後のプーリング層の出力のマップのサイズ=特徴量の数
        
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(500, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 10)
        
    def forward(self, x):
        #入力→畳み込み層1→活性化関数(ReLU)→プーリング層1(2*2)→出力
        # input 28 x 28 x 1
        # conv1 28 x 28 x 1 -> 24 x 24 x 10
        # max_pool(kernel2) 12 x 12 x 10
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2,2) )
        
        #入力→畳み込み層2→活性化関数(ReLU)→プーリング層2(2*2)→出力
        # conv2 12 x 12 x 10 -> 8 x 8 x 20
        # max_pool(kernel2) -> 4 x 4 x 20
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.conv2_drop(x)
        # output layer
        #x = x.view(-1, self.num_flat_features(x))
        # self.num_flat_featuresで特徴量の数を算出
        # flatten 4 x 4 x 20 = 320
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
#         x = F.log_softmax(x, dim=1)
        
        return x
    
    def num_flat_features(self, x):
        #Conv2dは入力を4階のテンソルとして保持する(サンプル数*チャネル数*縦の長さ*横の長さ)
        #よって、特徴量の数を数える時は[1:]でスライスしたものを用いる
        size = x.size()[1:] ## all dimensions except the batch dimension
        #特徴量の数=チャネル数*縦の長さ*横の長さを計算する
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [8]:
class TrainLogger(object):
    
    def __init__(self, out):
        try:
            os.makedirs(out)
        except OSError:
            pass
        self.file = open(os.path.join(out, 'log'), 'w')
        self.logs = []
        
    def write(self, log):
        ## write log
        tqdm.write(log)
        tqdm.write(log, file=self.file)
        self.logs.append(log)
        
    def state_dict(self):
        ## returns the state of the loggers
        return {'logs': self.logs}
    
    def load_state_dict(self, state_dict):
        ## load the logger state
        self.logs = state_dict['logs']
        #write logs
        tqdm.write(self.logs[-1])
        for log in self.logs:
            tqdm.write(log, file=self.file)
            

In [9]:
def checkpoint(net, optimizer, epoch, logger, out):
    filename = os.path.join(out, 'epoch-{}'.format(epoch))
    torch.save({'epoch': epoch + 1, 'logger': logger.state_dict()}, filename + '.iter')
    torch.save(net.state_dict(), filename + 'model')
    torch.save(optimizer.state_dict(), filename + 'state')

In [10]:
class InfiniteSampler(sampler.Sampler):

    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __iter__(self):
        while True:
            order = np.random.permutation(self.num_samples)
            for i in range(self.num_samples):
                yield order[i]

    def __len__(self):
        return None

In [11]:
class MNISTDataSet(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):

        self.image_dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.image_dataframe.loc[idx, 'img'])
#         image = io.imread(img_name)
        image = Image.open(img_name)
        image = image.convert('L')
        label = self.image_dataframe.loc[idx, 'label']

        if self.transform:
            image = self.transform(image)

        return image, label

In [12]:
def l2_normalize(d):
    ## 画像の場合、batch x (WxHxC) x 1 x 1
    ## この処理の意図は最終的にバッチごとにnormを計算するため
    d_reshape = d.view(d.shape[0], -1, *(1 for _ in range(d.dim() - 2)))
#     d_norm = torch.norm(d_reshape, dim=1, keepdim=True) + 1e-8
    
    d_norm = torch.sqrt(torch.sum(d_reshape**2, dim=1, keepdim=True))
    
    return d / d_norm

In [13]:
def kl_div(log_probs, probs):

#     qlogq = (probs * torch.log(probs)).sum()
#     qlogp = (probs * log_probs).sum()
#     return  (qlogq - qlogp).sum()
        
    return F.kl_div(log_probs, probs, reduction='sum')

#     return F.kl_div(log_probs, probs, reduction='elementwise_mean')
    

In [14]:
@contextlib.contextmanager
def _disable_tracking_bn_stats(model):

    def switch_attr(m):
        if hasattr(m, 'track_running_stats'):
            m.track_running_stats ^= True
            
    model.apply(switch_attr)
    yield
    model.apply(switch_attr)

In [15]:
class VAT(nn.Module):
    def __init__(self, xi=10.0, eps=1.0, ip=1):
        
        super(VAT, self).__init__()
        self.xi = xi
        self.eps = eps
        self.ip = ip
    
    def forward(self, model, x):
        
        ## ラベル無しデータをネットワークに通し、predictを得る
        ## VATのLossのbackpropagationでは、このmodel計算は含めないため
        ## no_gradでwrapする
        with torch.no_grad():
            out = model(x)
            pred = F.softmax(out, dim=1)
        
        ## 累積法を用いてVadvを計算する
        
        d = torch.rand(x.shape).sub(0.5).to(x.device) ## ゼロを中心に乱数による初期値
        d = l2_normalize(d)
        
        with _disable_tracking_bn_stats(model):
            for _ in range(self.ip):
                d.requires_grad_()
                pred_hat = model(x + self.xi * d)
                adv_dist = kl_div(F.log_softmax(pred_hat, dim=1), pred)
                
                adv_dist.backward()
                d = l2_normalize(d.grad)
                model.zero_grad()

        ## LDSを計算する
        r_adv = d * self.eps
        pred_hat = model(x + r_adv)
        lds = kl_div(F.log_softmax(pred_hat, dim=1), pred)
        
        return lds
    

In [16]:
an = np.random.rand(30).reshape(3,10)
an = an / (an.mean(axis=1).reshape(3,1))
an = torch.from_numpy(an)
bn = np.random.rand(30).reshape(3,10)
bn = bn / (bn.mean(axis=1).reshape(3,1))
bn = torch.from_numpy(bn)

In [17]:
## kl_divのlossの算出方法 -> reductionで設定
## reduction = 'elementwise_mean'
qlogq = (bn * torch.log(bn)).mean()
qlogp = (bn * torch.log(an)).mean()
print(qlogq - qlogp)

## reduction = 'sum' >> default
qlogq = (bn * torch.log(bn)).sum()
qlogp = (bn * torch.log(an)).sum()
print(qlogq - qlogp)

print(F.kl_div(torch.log(an), bn, reduction='elementwise_mean'))
print(F.kl_div(torch.log(an), bn, reduction='sum'))
print(F.kl_div(torch.log(an), bn, size_average=False))
print(F.kl_div(torch.log(an), bn))

tensor(0.3286, dtype=torch.float64)
tensor(9.8591, dtype=torch.float64)
tensor(0.3286, dtype=torch.float64)
tensor(9.8591, dtype=torch.float64)
tensor(9.8591, dtype=torch.float64)
tensor(0.3286, dtype=torch.float64)


/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [18]:
def test(model, device, test_loader, criterion, logger):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    log = 'Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, total, 100. * correct / total)
    logger.write(log)

In [19]:
# pick_data = []
def train(model, device, label_loader, unlabel_loader, testloader,criterion, vat, optimizer, epoch, logger, args):
    model.train()
    correct = 0
    total = 0
#     for batch_id, (data, target) in enumerate(train_loader):
#     label_iter = iter(label_loader)
#     unlabel_iter = iter(unlabel_loader)
    
    
    for iter_id in range(args.iters):

        label_data, label_target = next(label_loader)
        unlabel_data, _ = next(unlabel_loader)
    
#         label_data, label_target = label_iter.next()
#         unlabel_data, _ = unlabel_iter.next()
            
    
        label_data_size = label_data.size()[0]
        unlabel_data_size = unlabel_data.size()[0]
        
        
        label_data, label_target = label_data.to(device), label_target.to(device)
        unlabel_data = unlabel_data.to(device)       
                
        optimizer.zero_grad()
        label_output = model(label_data)
       
        label_loss = criterion(label_output, label_target)
#         lds_unlabel = vat(model, unlabel_data)
#         lds_label = vat(model, label_data)
        lds_unlabel = vat(model, unlabel_data) / unlabel_data_size
#         lds_label = vat(model, label_data) / label_data_size
        
        loss = label_loss + args.alpha * (lds_unlabel)

        loss.backward()
        optimizer.step()
        
        if iter_id % args.log_interval == 0:
            _, predicted = torch.max(label_output, 1)
            total += label_target.size(0)
            correct += (predicted == label_target).sum().item() 
            log = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.0f}%'.format(
                epoch,
                iter_id * len(label_data), args.iters * len(label_data),
                100. * (iter_id * len(label_data)) /(args.iters * len(label_data)),
                label_loss.item(),
                100. * correct / total)
            logger.write(log)
        
        if iter_id % (args.iters * 0.1) == 0:
            test(model, device, testloader, criterion, logger)
            model.train()
#             checkpoint(net, optimizer, epoch, logger, out_dir)
#             print('loss:        {}'.format(label_loss))
#             print('vat_loss_l:  {}'.format(lds_unlabel))
#             print('vat_loss_nl: {}'.format(lds_label))
           

In [20]:
def main():
    args = easydict.EasyDict({
        "label_batch_size": 100,
        "unlabel_batch_size": 200,
        "test_batch_size": 10,
        "epochs": 1,
        "iters": 20000,
        "lr": 0.01,
        "momentum": 0.9,
        "alpha": 1.0,
#         "xi": 1e-6,
        "xi": 25,
        "eps": 2.0,
        "ip": 1,
        "workers": 8,
        "seed": 123,
        "log_interval": 100,
        "gpu": 6
    })
    
   
    no_cuda = False
    out_dir = './result'
    train_label_csv = '../data/mnist/train_label.csv'
    train_unlabel_csv = '../data/mnist/train_big.csv'
    test_csv = '../data/mnist/test_small.csv'
    train_root_dir = '../data/mnist/train'
    test_root_dir = '../data/mnist/test'
    test_interval = 1
    resume_interval = 1
    
    use_cuda = not no_cuda and torch.cuda.is_available()    
    torch.manual_seed(args.seed)
    device = torch.device('cuda:{}'.format(args.gpu) if use_cuda else 'cpu')
    print(device)
    kwargs = {'num_workers':8, 'pin_memory': True} if use_cuda else {}
    
    transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5,))])
    
    trainset_label = MNISTDataSet(train_label_csv, train_root_dir, transform)    
    trainloader_label = torch.utils.data.DataLoader(
        trainset_label, batch_size=args.label_batch_size,
        sampler=InfiniteSampler(len(trainset_label)),
        **kwargs
    )

    trainset_unlabel = MNISTDataSet(train_unlabel_csv, train_root_dir, transform)    
    trainloader_unlabel = torch.utils.data.DataLoader(
        trainset_unlabel, batch_size=args.unlabel_batch_size,
        sampler=InfiniteSampler(len(trainset_unlabel)),
        **kwargs
    )

    testset = MNISTDataSet(test_csv, test_root_dir, transform)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=args.test_batch_size, shuffle=False, **kwargs
    )
    
    net = Net2().to(device)
#     net = CNN().to(device)
#     optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=0.25)
    
    optimizer = optim.Adam(net.parameters())
    criterion = nn.CrossEntropyLoss()
    vat = VAT(args.xi, args.eps, args.ip)
    logger = TrainLogger(out_dir)
    
    data_iterators = get_iters(
        root_path='.',
        l_batch_size=args.label_batch_size,
        ul_batch_size=args.unlabel_batch_size,
        test_batch_size=args.test_batch_size,
        workers=args.workers
    )
    
    trainloader_label = data_iterators['labeled']
    trainloader_unlabel = data_iterators['unlabeled']
    testloader = data_iterators['test']
    
#     for epoch in range(1, args.epochs + 1):
    epoch = 1
    train(net, device, trainloader_label, trainloader_unlabel, testloader,criterion, vat, optimizer, epoch, logger, args)
    test(net, device, testloader, criterion, logger)
    
#         if epoch % test_interval == 0:
#             test(net, device, testloader, criterion, logger)
#         if epoch % resume_interval == 0:
#             checkpoint(net, optimizer, epoch, logger, out_dir)

In [21]:
%ls ../../pytorch/data/mnist/

test/         test_small.csv  train_big.csv    train_small.csv
test_big.csv  train/          train_label.csv  train_unlabel.csv


In [22]:
df_train = pd.read_csv('../../pytorch/data/mnist/train_big.csv')
df_test = pd.read_csv('../../pytorch/data/mnist/test_big.csv')

In [23]:
df_test.head()

,img,label
0,0/0_00000.png,0
1,0/0_00001.png,0
2,0/0_00002.png,0
3,0/0_00003.png,0
4,0/0_00004.png,0


In [24]:
n_labeled = 100
valid_size = 1000
x_train, y_train = np.array(df_train['img']), np.array(df_train['label'])
x_test, y_test =  np.array(df_test['img']), np.array(df_test['label'])

randperm = np.random.permutation(len(x_train))
labeled_idx = randperm[:n_labeled]
validation_idx = randperm[n_labeled:n_labeled + valid_size]
unlabeled_idx = randperm[n_labeled + valid_size:]

x_labeled = x_train[labeled_idx]
x_validation = x_train[validation_idx]
x_unlabeled = x_train[unlabeled_idx]

y_labeled = y_train[labeled_idx]
y_validation = y_train[validation_idx]

In [25]:
path = '../../pytorch/data/mnist/train'
img_path = os.path.join(path, x_labeled[0])
image = Image.open(img_path)
# image = image.convert('L')

In [26]:
torch.Tensor(np.array(image)).size()

torch.Size([28, 28])

In [27]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms

class SimpleDataset2(Dataset):

    def __init__(self, x, y, transform,root):
        self.x = x
        self.y = y
        self.transform = transform
        self.root = root

    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.x[index])
        img = np.array(Image.open(img_path))
        img = torch.Tensor(img)
        img = img.view([1,28,28])
        if self.transform is not None:
            img = self.transform(img)
        target = self.y[index]
        return img, target

    def __len__(self):
        return len(self.x)


class InfiniteSampler(sampler.Sampler):

    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __iter__(self):
        while True:
            order = np.random.permutation(self.num_samples)
            for i in range(self.num_samples):
                yield order[i]

    def __len__(self):
        return None


def get_iters2(
        dataset='mnist', root_path='.', data_transforms=None,
        n_labeled=100, valid_size=1000,
        l_batch_size=32, ul_batch_size=128, test_batch_size=256,
        workers=8, pseudo_label=None):
    
    # train_path = f'{root_path}/data/{dataset}/train/'
    # test_path = f'{root_path}/data/{dataset}/test/'
    
    train_root = '../../pytorch/data/mnist/train'
    test_root = '../../pytorch/data/mnist/test'
    df_train = pd.read_csv('../../pytorch/data/mnist/train_big.csv')
    df_test = pd.read_csv('../../pytorch/data/mnist/test_big.csv')
    
  
    if data_transforms is None:
        data_transforms = {
            'train': transforms.Compose([
                transforms.ToPILImage(),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
                # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
            'eval': transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
                # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
        }

    # print(train_dataset.train_data)

    x_train, y_train = np.array(df_train['img']), np.array(df_train['label'])
    x_test, y_test =  np.array(df_test['img']), np.array(df_test['label'])

    randperm = np.random.permutation(len(x_train))
    labeled_idx = randperm[:n_labeled]
    validation_idx = randperm[n_labeled:n_labeled + valid_size]
    unlabeled_idx = randperm[n_labeled + valid_size:]

    x_labeled = x_train[labeled_idx]
    x_validation = x_train[validation_idx]
    x_unlabeled = x_train[unlabeled_idx]

    y_labeled = y_train[labeled_idx]
    y_validation = y_train[validation_idx]
    if pseudo_label is None:
        y_unlabeled = y_train[unlabeled_idx]
    else:
        assert isinstance(pseudo_label, np.ndarray)
        y_unlabeled = pseudo_label


    data_iterators = {
        'labeled': iter(DataLoader(
            SimpleDataset2(x_labeled, y_labeled, data_transforms['train'], train_root),
            batch_size=l_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_labeled)),
        )),
        'unlabeled': iter(DataLoader(
            SimpleDataset2(x_unlabeled, y_unlabeled, data_transforms['train'], train_root),
            batch_size=ul_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_unlabeled)),
        )),
        'make_pl': iter(DataLoader(
            SimpleDataset2(x_unlabeled, y_unlabeled, data_transforms['eval'], train_root),
            batch_size=ul_batch_size, num_workers=workers, shuffle=False
        )),
        'val': iter(DataLoader(
            SimpleDataset2(x_validation, y_validation, data_transforms['eval'], train_root),
            batch_size=len(x_validation), num_workers=workers, shuffle=False
        )),
        'test': DataLoader(
            SimpleDataset2(x_test, y_test, data_transforms['eval'], test_root),
            batch_size=test_batch_size, num_workers=workers, shuffle=False
        )
    }

    return data_iterators

In [28]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import sampler
from torchvision import datasets
from torchvision import transforms

class SimpleDataset(Dataset):

    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform

    def __getitem__(self, index):
        img = self.x[index]
        img = img.view([1,28,28])
        if self.transform is not None:
            img = self.transform(img)
        target = self.y[index]
        return img, target

    def __len__(self):
        return len(self.x)


class InfiniteSampler(sampler.Sampler):

    def __init__(self, num_samples):
        self.num_samples = num_samples

    def __iter__(self):
        while True:
            order = np.random.permutation(self.num_samples)
            for i in range(self.num_samples):
                yield order[i]

    def __len__(self):
        return None


def get_iters(
        dataset='mnist', root_path='.', data_transforms=None,
        n_labeled=100, valid_size=1000,
        l_batch_size=32, ul_batch_size=128, test_batch_size=256,
        workers=8, pseudo_label=None):
    
    # train_path = f'{root_path}/data/{dataset}/train/'
    # test_path = f'{root_path}/data/{dataset}/test/'

    train_path = '../../pytorch/data/'
    test_path = '../../pytorch/data/'
    
    
    

    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    if dataset == 'CIFAR10':
        train_dataset = datasets.CIFAR10(train_path, download=True, train=True, transform=None)
        test_dataset = datasets.CIFAR10(test_path, download=True, train=False, transform=None)
    elif dataset == 'CIFAR100':
        train_dataset = datasets.CIFAR100(train_path, download=True, train=True, transform=None)
        test_dataset = datasets.CIFAR100(test_path, download=True, train=False, transform=None)
    elif dataset == 'mnist':
        print('mnist')
        train_dataset = datasets.MNIST(root=train_path, train=True, download=True,transform=transform)
        test_dataset = datasets.MNIST(root=test_path, train=False, download=False,transform=transform)
    else:
        raise ValueError

    if data_transforms is None:
        data_transforms = {
            'train': transforms.Compose([
                transforms.ToPILImage(),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
                # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
            'eval': transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
                # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
            ]),
        }

    # print(train_dataset.train_data)

    x_train, y_train = train_dataset.train_data, np.array(train_dataset.train_labels)
    x_test, y_test = test_dataset.test_data, np.array(test_dataset.test_labels)

    randperm = np.random.permutation(len(x_train))
    labeled_idx = randperm[:n_labeled]
    validation_idx = randperm[n_labeled:n_labeled + valid_size]
    unlabeled_idx = randperm[n_labeled + valid_size:]

    x_labeled = x_train[labeled_idx]
    x_validation = x_train[validation_idx]
    x_unlabeled = x_train[unlabeled_idx]

    y_labeled = y_train[labeled_idx]
    y_validation = y_train[validation_idx]
    if pseudo_label is None:
        y_unlabeled = y_train[unlabeled_idx]
    else:
        assert isinstance(pseudo_label, np.ndarray)
        y_unlabeled = pseudo_label


    data_iterators = {
        'labeled': iter(DataLoader(
            SimpleDataset(x_labeled, y_labeled, data_transforms['train']),
            batch_size=l_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_labeled)),
        )),
        'unlabeled': iter(DataLoader(
            SimpleDataset(x_unlabeled, y_unlabeled, data_transforms['train']),
            batch_size=ul_batch_size, num_workers=workers,
            sampler=InfiniteSampler(len(x_unlabeled)),
        )),
        'make_pl': iter(DataLoader(
            SimpleDataset(x_unlabeled, y_unlabeled, data_transforms['eval']),
            batch_size=ul_batch_size, num_workers=workers, shuffle=False
        )),
        'val': iter(DataLoader(
            SimpleDataset(x_validation, y_validation, data_transforms['eval']),
            batch_size=len(x_validation), num_workers=workers, shuffle=False
        )),
        'test': DataLoader(
            SimpleDataset(x_test, y_test, data_transforms['eval']),
            batch_size=test_batch_size, num_workers=workers, shuffle=False
        )
    }

    return data_iterators

In [29]:
main()

cuda:6
mnist
Train Epoch: 1 [0/2000000 (0%)]	Loss: 2.302507	Accuracy: 10%
Test set: Average loss: 2233.5180, Accuracy: 4853/10000 (49%)
Train Epoch: 1 [10000/2000000 (0%)]	Loss: 0.021697	Accuracy: 55%
Train Epoch: 1 [20000/2000000 (1%)]	Loss: 0.011272	Accuracy: 70%
Train Epoch: 1 [30000/2000000 (2%)]	Loss: 0.008089	Accuracy: 78%
Train Epoch: 1 [40000/2000000 (2%)]	Loss: 0.005414	Accuracy: 82%
Train Epoch: 1 [50000/2000000 (2%)]	Loss: 0.005644	Accuracy: 85%
Train Epoch: 1 [60000/2000000 (3%)]	Loss: 0.004548	Accuracy: 87%
Train Epoch: 1 [70000/2000000 (4%)]	Loss: 0.003724	Accuracy: 89%
Train Epoch: 1 [80000/2000000 (4%)]	Loss: 0.003088	Accuracy: 90%
Train Epoch: 1 [90000/2000000 (4%)]	Loss: 0.002744	Accuracy: 91%
Train Epoch: 1 [100000/2000000 (5%)]	Loss: 0.002144	Accuracy: 92%
Train Epoch: 1 [110000/2000000 (6%)]	Loss: 0.001905	Accuracy: 92%
Train Epoch: 1 [120000/2000000 (6%)]	Loss: 0.001986	Accuracy: 93%
Train Epoch: 1 [130000/2000000 (6%)]	Loss: 0.001763	Accuracy: 94%
Train Epoch: 1 

Train Epoch: 1 [1170000/2000000 (58%)]	Loss: 0.000206	Accuracy: 99%
Train Epoch: 1 [1180000/2000000 (59%)]	Loss: 0.000205	Accuracy: 99%
Train Epoch: 1 [1190000/2000000 (60%)]	Loss: 0.000193	Accuracy: 99%
Train Epoch: 1 [1200000/2000000 (60%)]	Loss: 0.000288	Accuracy: 99%
Test set: Average loss: 53.8434, Accuracy: 9835/10000 (98%)
Train Epoch: 1 [1210000/2000000 (60%)]	Loss: 0.000183	Accuracy: 99%
Train Epoch: 1 [1220000/2000000 (61%)]	Loss: 0.000190	Accuracy: 99%
Train Epoch: 1 [1230000/2000000 (62%)]	Loss: 0.000192	Accuracy: 99%
Train Epoch: 1 [1240000/2000000 (62%)]	Loss: 0.000200	Accuracy: 99%
Train Epoch: 1 [1250000/2000000 (62%)]	Loss: 0.000204	Accuracy: 99%
Train Epoch: 1 [1260000/2000000 (63%)]	Loss: 0.000199	Accuracy: 99%
Train Epoch: 1 [1270000/2000000 (64%)]	Loss: 0.000168	Accuracy: 99%
Train Epoch: 1 [1280000/2000000 (64%)]	Loss: 0.000169	Accuracy: 99%
Train Epoch: 1 [1290000/2000000 (64%)]	Loss: 0.000196	Accuracy: 99%
Train Epoch: 1 [1300000/2000000 (65%)]	Loss: 0.000201	Ac

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x2aeaa7b44898>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 345, in get
    return ForkingPickler.loads(res)
  File "/usr/local/lib/python3.5/dist-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/usr/lib/python3.5/multiprocessing/reduction.py", line 181, in recv_handle
    return recvfds(s, 1)[0]
  File "/usr/lib/python3.5/multiprocessing/reduction.py", line 152, in rec

In [30]:
%ls ../data/mnist/

test/         test_small.csv  train_big.csv    train_small.csv
test_big.csv  train/          train_label.csv  train_unlabel.csv


In [24]:
torch.__version__

'0.4.1'